In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install kaggle
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 24.6 MB/s eta 0:00:00


In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import json
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi
import pymongo

# connect to mongodb client, did not write explicitly due to confidential reasons
client = pymongo.MongoClient("")
db = client['Crawl-Data']
collection = db['metadata']

In [ ]:
kaggle_dt = {}
# !mkdir .kaggle
# !mv /.kaggle /root/

In [ ]:
# For this part, you need to download kaggle.json from kaggle user settings/API and upload to google colab.
# When done uploading, use !mkdir .kaggle to make a .kaggle folder to contain the json, and use !mv /.kaggle /root/ to move the folder to root directory
# Then this block of code will run
# Instantiate the Kaggle API
api = KaggleApi()
#os.environ['KAGGLE_CONFIG_DIR'] = '/content'
api.authenticate()
# Fetch the list of datasets, now we are fetching 10000, 20 per page
count = 0
for i in range(1,501):
  datasets = api.dataset_list(page=i)
  for dataset in datasets:
      #need to download the metadatafile and read it each time to get information
      try:
        metadata = api.dataset_metadata(dataset.ref, path = '/content/metadata.json')
      except:
        continue
      with open('/content/metadata.json/dataset-metadata.json') as metadata_file:
            metadata = json.load(metadata_file)
            metadata['name'] = dataset.ref
            metadata['url'] = dataset.url
            #add Kaggle data to mongodb
            existing_data = collection.find_one({"url": dataset.url})
            if existing_data is None:
              # Data is not in the collection, so insert it
              collection.insert_one(metadata)
              count += 1


print('Inserted', count, "number of data")
'''
        print("Name:", dataset.ref)
        print("URL:", dataset.url)
        print()
      kaggle_dt[dataset.ref] = dataset.url
      '''
'''
# Print dataset names and URLs
with open('/content/drive/MyDrive/Kaggle_Output_Reference.txt', 'w') as f:
  f.write(json.dumps(kaggle_dt))
'''

OSError: ignored

In [5]:
# get all dataset urls on UCI - 607 in total

urls = set()
URL = "https://archive.ics.uci.edu/datasets?skip=0&take=1000&sort=desc&orderBy=NumHits&search=" # assuming uci archive size no more than 1k
page = requests.get(URL)
soup = bs(page.content, "html.parser")

# find number of datasets
# size = int(soup.find("div", class_="my-4 flex flex-wrap items-center justify-end gap-4").find_all("span")[-1].text) + 1

for h2 in soup.find_all('h2'):
    urls.add(h2.find('a')['href'])

In [6]:
# build dt: dict of txt name - url pairs

dt = pd.DataFrame(columns = ["id", "url"])
for url in tqdm(list(urls)):
    dt.loc[len(dt)] = [url[url.rfind("/")+1:], "https://archive.ics.uci.edu" + url]

# write to output_reference.txt
'''
with open('/content/drive/MyDrive/colai/crawler/Output_Reference_v2.txt', 'w') as f:
     f.write(json.dumps(dt))
'''
dt

100%|██████████| 644/644 [00:02<00:00, 277.65it/s]


,id,url
0,statlog+landsat+satellite,https://archive.ics.uci.edu/dataset/146/statlo...
1,insurance+company+benchmark+coil+2000,https://archive.ics.uci.edu/dataset/125/insura...
2,statlog+image+segmentation,https://archive.ics.uci.edu/dataset/147/statlo...
3,australian+sign+language+signs,https://archive.ics.uci.edu/dataset/114/austra...
4,ubiqlog+smartphone+lifelogging,https://archive.ics.uci.edu/dataset/369/ubiqlo...
...,...,...
639,el+nino,https://archive.ics.uci.edu/dataset/122/el+nino
640,csm+conventional+and+social+media+movies+datas...,https://archive.ics.uci.edu/dataset/424/csm+co...
641,data+for+software+engineering+teamwork+assessm...,https://archive.ics.uci.edu/dataset/393/data+f...
642,opinrank+review+dataset,https://archive.ics.uci.edu/dataset/205/opinra...


In [7]:
# identify variables to include in the table

page0 = requests.get("https://archive.ics.uci.edu" + list(urls)[0])
soup0 = bs(page0.text, "html.parser")
tb = pd.DataFrame(columns = ["Title","Description"] + [i.text for i in soup0.find_all('h1')][1:7])

for url in tqdm(list(urls)):
    page0 = requests.get("https://archive.ics.uci.edu" + url)
    soup0 = bs(page0.text, "html.parser")
    txt2 = soup0.find_all(['h1','p'])
    tb.loc[len(tb)] = [soup0.find('h1').text] + [i.text for i in soup0.find_all('p')][:7]
    '''
    with open('/content/drive/MyDrive/colai/crawler/outputs_v2/{}.txt'.format(url[url.rfind("/")+1:]), "w") as f:
        for line in txt2:
            if line.text == "Creators":
                break
            else:
                f.write(line.text)
                f.write('\n')
    '''


100%|██████████| 644/644 [06:11<00:00,  1.73it/s]


In [8]:
df_final = pd.concat([dt, tb], axis = 1)
df_final = json.loads(df_final.to_json(orient = "records"))
df_final

[{'id': 'statlog+landsat+satellite',
  'url': 'https://archive.ics.uci.edu/dataset/146/statlog+landsat+satellite',
  'Title': 'Statlog (Landsat Satellite)',
  'Description': ' Multi-spectral values of pixels in 3x3 neighbourhoods in a satellite image, and the classification associated with the central pixel in each neighbourhood',
  'Dataset Characteristics': 'Multivariate',
  'Subject Area': 'Physical Science',
  'Associated Tasks': 'Classification',
  'Attribute Type': 'Integer',
  '# Instances': '6435',
  '# Attributes': '36'},
 {'id': 'insurance+company+benchmark+coil+2000',
  'url': 'https://archive.ics.uci.edu/dataset/125/insurance+company+benchmark+coil+2000',
  'Title': 'Insurance Company Benchmark (COIL 2000)',
  'Description': ' This data set used in the CoIL 2000 Challenge contains information on customers of an insurance company. The data consists of 86 variables and includes product usage data and socio-demographic data',
  'Dataset Characteristics': 'Multivariate',
  'Sub

In [9]:
# add UCI data to mongodb
count = 0
for data in df_final:
    # exists duplicate names, but different datasets
    existing_data = collection.find_one({"url": data["url"]})
    if existing_data is None:
        # Data is not in the collection, so insert it
        collection.insert_one(data)
        count += 1
print('Inserted', count, "number of data")

Inserted 644 number of data


In [ ]:
len(df_final)

624